In [1]:
import os 

os.chdir("app/")

In [2]:
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.trainer import Trainer

from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load.datasets import MemoryMapDataset
from ptls.preprocessing import PandasDataPreprocessor

from ptls.nn import TrxEncoder
from ptls.nn.seq_encoder import RnnSeqEncoder

from ptls.frames import PtlsDataModule
from ptls.frames.coles.coles_module import CoLESModule
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices

from utils.encode import encode_data
from utils.evaluation import bootstrap_eval

## Load data

In [3]:
df = pd.read_parquet("data/preprocessed_new/age.parquet")
df.head()

,user_id,timestamp,mcc_code,amount,global_target
0,33172,6,4,71.463,0
1,33172,6,35,45.017,0
2,33172,8,11,13.887,0
3,33172,9,11,15.983,0
4,33172,10,11,21.341,0


## Prepare dataset and dataloaders

In [4]:
preprocessor = PandasDataPreprocessor(
    col_id="user_id",
    col_event_time="timestamp",
    event_time_transformation="none",
    cols_category=["mcc_code"],
    cols_first_item=["global_target"]
)

In [5]:
data = preprocessor.fit_transform(df)

In [6]:
val_size = 0.1
test_size = 0.1

train, val_test = train_test_split(data, test_size=test_size+val_size, random_state=42)
val, test = train_test_split(val_test, test_size=test_size/(test_size+val_size), random_state=42)

In [7]:
train_ds = ColesDataset(
    data=MemoryMapDataset(train, [SeqLenFilter(min_seq_len=15)]),
    splitter=SampleSlices(5, 25, 200),
)
val_ds = ColesDataset(
    data=MemoryMapDataset(val, [SeqLenFilter(min_seq_len=15)]),
    splitter=SampleSlices(5, 15, 200),
)
test_ds = MemoryMapDataset(test, [SeqLenFilter(min_seq_len=15)])

In [8]:
datamodule = PtlsDataModule(
    train_data=train_ds,
    valid_data=val_ds,
    train_batch_size=512,
    valid_batch_size=512,
    train_num_workers=16,
    valid_num_workers=16
)

# CoLES

## Model training

In [9]:
trx_encoder = TrxEncoder(
    embeddings={
        "mcc_code": {"in": 250, "out": 16}
    },
    numeric_values={
        "amount": "identity",
    },
    use_batch_norm_with_lens=True,
    norm_embeddings=False,
    embeddings_noise=0.003
)

seq_encoder = RnnSeqEncoder(
    trx_encoder,
    hidden_size=800,
    type="gru",
    bidir=False,
    trainable_starter="static",
)

In [10]:
lr_scheduler_partial = partial(torch.optim.lr_scheduler.ReduceLROnPlateau, factor=.9025, patience=5, mode="max")
optimizer_partial = partial(torch.optim.Adam, lr=1e-3)

model = CoLESModule(
    seq_encoder,
    optimizer_partial=optimizer_partial,
    lr_scheduler_partial=lr_scheduler_partial
)

In [11]:
checkpoint = ModelCheckpoint(
    monitor="recall_top_k", 
    mode="max"
)

trainer = Trainer(
    max_epochs=150,
    devices=[0],
    accelerator="gpu",
    callbacks=[checkpoint]
)

trainer.fit(model, datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
model.load_state_dict(torch.load(checkpoint.best_model_path)["state_dict"])

## Model evaluation

In [13]:
train_val_ds = MemoryMapDataset(train + val, [SeqLenFilter(min_seq_len=15)])

In [14]:
X_train, y_train = encode_data(model.seq_encoder, train_val_ds)
X_test, y_test = encode_data(model.seq_encoder, test_ds)

print("Train size:", len(y_train))
print("Test size:", len(y_test))

Train size: 27000
Test size: 3000


In [15]:
results = bootstrap_eval(X_train, X_test, y_train, y_test, n_runs=10)

100%|██████████| 10/10 [06:57<00:00, 41.79s/it]


In [18]:
results

,ROC-AUC,PR-AUC,Accuracy
0,0.859987,0.678424,0.628333
1,0.860392,0.673144,0.625667
2,0.859904,0.677720,0.627667
3,0.858270,0.671257,0.630000
4,0.858597,0.674139,0.626667
5,0.861548,0.679126,0.637333
6,0.859171,0.676463,0.629667
7,0.859807,0.675470,0.636667
8,0.858775,0.671772,0.633000
9,0.858263,0.671024,0.629667


In [17]:
results.agg(["mean", "std"])

,ROC-AUC,PR-AUC,Accuracy
mean,0.859471,0.674854,0.630467
std,0.001052,0.003032,0.003991
